In [1]:
from twitter import *
import requests
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
from matplotlib.path import Path
import shapefile

In [2]:
class TwitterCustom(object):
    def __init__(self):
        consumer_key = 'LFTXnbNVhCtLV2T2dG0yA'
        consumer_secret = '2DX6L81tUHP8Clxz8FvuXcaMvHTAXuUkXbzz7y6U'
        access_token = '77452634-avWq03PwpORlOtKoa4ctg082fTdFlBbKT0ihC8XZw'
        access_token_secret = 'cnopinzlsPRi6zBOMGdAVf0EuMM97eCQF5UPoyTyY'

        self.api = Twitter(auth=OAuth(access_token, access_token_secret, consumer_key, consumer_secret))
        
    def get_tweet(self, id):
        return self.api.statuses.show(_id=724694725601669120)

In [84]:
def get_geoname_data(geoname):
    return requests.get(u'http://api.geonames.org/postalCodeSearchJSON?placename={0}&maxRows=100&username=demo&style=FULL'.format(geoname)).json()

In [73]:
def make_graph(polygones):
    fig = plt.figure()
    ax = Axes3D(fig)
    
    # Make path
    paths = [Path([[p[0], p[1]] for p in poly]) for poly in polygones]
    
    x = []
    y = []
    z = []
    for poly in polygones:
        for point in poly:
            x.append(point[0])
            y.append(point[1])
            z_temp = point[2]
            
            for i in range(0, len(polygones) -1):
                if not polygones[i] == poly:
                    if paths[i].contains_point([point[0], point[1]]):
                        z_temp += poly2[0][2]
            z.append(z_temp)
        
    ax.plot_trisurf(x, y, z)
        
    ax.set_xlim3d(min(x), max(x))
    ax.set_ylim3d(min(y), max(y))
    ax.set_zlim3d(min(z), max(z))
    plt.show()

In [26]:
def get_time_sone_area(tweet):
    # Read shape file: SLOW OPERATION!!
    tz = shapefile.Reader("data/tz/tz_world_mp")
    records = tz.records()
    
    tweet_tz = tweet['user']['time_zone']
    index_tz = 0
    for name in records:
        if tweet_tz in name[0]:
            break
        index_tz+=1
    
    return tz.shape(index_tz).points

In [41]:
def add_z(poly, z):
    for p in poly:
        p.append(z)
    return poly

In [87]:
def determinate_tweet_localisation(tweet):
    # Get area by geoname in tweet
    poly_tz = add_z(get_time_sone_area(tweet), 1)
    
    # Get area by timezone
    
    # Get area by user localisation in profile
    
    # Get area by user bio
    
    return [poly_tz]

In [88]:
def get_tweet_localisation(tweet_id):
    t = TwitterCustom()
    polys = determinate_tweet_localisation(t.get_tweet(tweet_id))
    make_graph(polys)

In [89]:
get_tweet_localisation('')

/usr/lib/python2.7/dist-packages/mpl_toolkits/mplot3d/axes3d.py:704: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=1.0, top=1.0
  'bottom=%s, top=%s') % (bottom, top))


AttributeError: 'NoneType' object has no attribute 'tk'